In [19]:
import matplotlib.pyplot as plt
import pandas as pd 
import numpy as np
from sklearn.metrics import roc_curve, auc
import os

In [23]:
current_dir = os.getcwd()


fnn_output = pd.read_csv(os.path.normpath(os.path.join(current_dir, '../results/', 'fnn_output.csv')), sep='\t')
hnn_output = pd.read_csv(os.path.normpath(os.path.join(current_dir, '../results/', 'hmm_output.csv')), sep='\t')

lr_output = pd.read_csv(os.path.normpath(os.path.join(current_dir, '../results/', 'lr_output.csv')), sep='\t')
xgboost_output = pd.read_csv(os.path.normpath(os.path.join(current_dir, '../results/', 'xgboost.csv')), sep='\t')



In [28]:
lr_output

,p_recall,half_life,p_recall_pred,half_life_pred,slp,slh,outcome,decile
0,0.9999,274.0,0.000100,0.010417,0.999600,75070.291775,1.0,5
1,0.9999,274.0,0.000100,0.010417,0.999600,75070.291775,1.0,5
2,0.9999,274.0,0.799221,0.010417,0.040272,75070.291775,1.0,2
3,0.9999,274.0,0.799221,0.010417,0.040272,75070.291775,1.0,2
4,0.9999,274.0,0.799221,0.010417,0.040272,75070.291775,1.0,2
...,...,...,...,...,...,...,...,...
4616684,0.0001,274.0,0.000100,0.010417,0.000000,75070.291775,0.0,10
4616685,0.9999,274.0,0.000100,0.010417,0.999600,75070.291775,1.0,10
4616686,0.5000,274.0,0.744552,0.010417,0.059806,75070.291775,1.0,3
4616687,0.5000,274.0,0.744552,0.010417,0.059806,75070.291775,0.0,3


In [33]:
xgboost_output = xgboost_output.rename(columns={"y_test": "p_recall", "y_pred": "p_recall_pred"}, errors="raise")


In [34]:
datasets = [fnn_output, hnn_output, lr_output, xgboost_output]
labels = ["fnn", "hnn", "lr", "xgboost"] 


In [35]:
def AUC_ROC(datasets, labels, filename):
    
    plt.figure(figsize=(8, 6))
    
    for df, label in zip(datasets, labels):

        outcome = df['outcome']
        outcome_prob = df['p_recall_pred']
        
        # Calculate ROC curve and AUC
        fpr, tpr, _ = roc_curve(outcome, outcome_prob)
        roc_auc = auc(fpr, tpr)
        
        # Plot ROC curve
        plt.plot(fpr, tpr, label=f"{label} (AUC = {roc_auc:.3f})", linewidth=2)
        print(f"{label} - AUC: {roc_auc:.4f}")
    
    # Plot random guess line
    plt.plot([0, 1], [0, 1], 'k--', label="Random (AUC = 0.500)")
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC Curve Comparison')
    plt.legend(loc='lower right')
    plt.grid(True)
    
    plt.savefig(filename, bbox_inches='tight', dpi=300)
    plt.close()

In [36]:
AUC_ROC(datasets, labels, "roc_comparison.png")

fnn - AUC: 0.6014
hnn - AUC: 0.5320
lr - AUC: 0.5283
xgboost - AUC: 0.6548
